In [34]:
import streamlit as st
from dotenv import load_dotenv
import os
import google.generativeai as genai
from youtube_transcript_api import YouTubeTranscriptApi
from gtts import gTTS
from textblob import TextBlob
import nltk
import re
import logging

In [5]:
load_dotenv()

True

In [7]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [8]:
# Prompt for summarization
prompt = """
You are a YouTube video summarizer. Summarize the transcript text
in a concise way, providing important points within 350 words.
Here is the text to summarize:
"""

In [9]:
# Function to extract transcript from a YouTube video
def extract_transcript_details(youtube_video_url):
    try:
        video_id = youtube_video_url.split("v=")[1]
        transcript_text = YouTubeTranscriptApi.get_transcript(video_id)

        transcript = ""
        for item in transcript_text:
            transcript += " " + item["text"]

        return transcript
    except Exception as e:
        st.error(f"Error extracting transcript: {str(e)}")
        return None


In [12]:
transcript_text = extract_transcript_details('https://www.youtube.com/watch?v=KGkWFuk9xUE')

In [ ]:


def generate_gemini_content(transcript_text):
    try:
        llm = ChatGroq(
        api_key=GROQ_API_KEY,
        model_name="llama-3.1-70b-versatile",
        temperature=0.2
    )
        response = model.generate_content(prompt + transcript_text)
        return response.text
    except Exception as e:
        st.error(f"Error generating summary: {str(e)}")
        return None

In [22]:
def generate_gemini_content(transcript_text):
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")  # Replace with available model
        response = model.generate_content(prompt + transcript_text)
        # print(response)
        return response.text
    except Exception as e:
        st.error(f"Error generating summary: {str(e)}")
        return None

In [23]:
summary_text = generate_gemini_content(transcript_text=transcript_text)

In [25]:
def generate_audio(summary_text):
    try:
        tts = gTTS(summary_text)
        audio_file = "summary_audio.mp3"
        tts.save(audio_file)
        return audio_file
    except Exception as e:
        st.error(f"Error generating audio: {str(e)}")
        return None

In [26]:
generate_audio(summary_text=summary_text)

KeyboardInterrupt: 

In [40]:
# Function to perform sentiment analysis
def analyze_sentiment(summary_text):
    try:
        analysis = TextBlob(summary_text)
        return {
            "polarity": analysis.polarity,
            "subjectivity": analysis.subjectivity
        }
    except Exception as e:
        st.error(f"Error analyzing sentiment: {str(e)}")
        return None


In [42]:
def analyze_sentiment2(summary_text):
    """
    Analyzes the sentiment of the provided text and returns human-friendly interpretations.
    
    Args:
        summary_text (str): The summarized text or transcript to analyze
        
    Returns:
        dict: A dictionary containing sentiment analysis results in user-friendly format
    """
    try:
        # Perform basic sentiment analysis
        analysis = TextBlob(summary_text)
        
        # Interpret polarity (sentiment)
        if analysis.polarity > 0.5:
            sentiment = "Very positive"
        elif analysis.polarity > 0.1:
            sentiment = "Somewhat positive"
        elif analysis.polarity > -0.1:
            sentiment = "Neutral"
        elif analysis.polarity > -0.5:
            sentiment = "Somewhat negative"
        else:
            sentiment = "Very negative"
            
        # Interpret subjectivity
        if analysis.subjectivity > 0.7:
            objectivity = "Highly subjective/opinionated"
        elif analysis.subjectivity > 0.4:
            objectivity = "Somewhat subjective"
        elif analysis.subjectivity > 0.2:
            objectivity = "Fairly balanced"
        else:
            objectivity = "Highly objective/factual"
        
        # Generate a brief explanation
        explanation = f"This video appears to be {sentiment.lower()} in tone and {objectivity.lower()} in content."
        
        # # Identify key emotional words
        # emotional_words = []
        # for word, pos in analysis.tags:
        #     # Focus on adjectives, adverbs, and some verbs
        #     if pos.startswith('JJ') or pos.startswith('RB') or pos in ['VBD', 'VBG']:
        #         word_blob = TextBlob(word)
        #         if abs(word_blob.sentiment.polarity) > 0.3:
        #             emotional_words.append(word)
        
        # emotional_words = emotional_words[:5]  # Limit to top 5 emotional words
        
        # Create a user-friendly response
        result = {
            "sentiment": sentiment,
            "objectivity": objectivity,
            "explanation": explanation,
            # "emotional_words": emotional_words if emotional_words else ["No strong emotional words detected"],
            "technical": {
                "polarity": round(analysis.polarity, 2),
                "subjectivity": round(analysis.subjectivity, 2)
            }
        }
        
        return result
    except Exception as e:
        print(f"Error analyzing sentiment: {str(e)}")
        return None

In [39]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def analyze_sentiment3(summary_text):
    """
    Analyzes the sentiment of the provided text with proper error handling
    and a fallback method using basic pattern matching if TextBlob fails.
    
    Args:
        summary_text (str): The summarized text or transcript to analyze
        
    Returns:
        dict: A dictionary containing sentiment analysis results in user-friendly format
    """
    # First try to download necessary NLTK data if not already present
    try:
        # Try to ensure the required NLTK data is available
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            logging.info("Downloading required NLTK data...")
            nltk.download('punkt')
            
        try:
            nltk.data.find('corpora/brown')
        except LookupError:
            logging.info("Downloading additional required NLTK data...")
            nltk.download('brown')
            
        # Perform TextBlob analysis
        analysis = TextBlob(summary_text)
        
        # Interpret polarity (sentiment)
        if analysis.polarity > 0.5:
            sentiment = "Very positive"
        elif analysis.polarity > 0.1:
            sentiment = "Somewhat positive"
        elif analysis.polarity > -0.1:
            sentiment = "Neutral"
        elif analysis.polarity > -0.5:
            sentiment = "Somewhat negative"
        else:
            sentiment = "Very negative"
            
        # Interpret subjectivity
        if analysis.subjectivity > 0.7:
            objectivity = "Highly subjective/opinionated"
        elif analysis.subjectivity > 0.4:
            objectivity = "Somewhat subjective"
        elif analysis.subjectivity > 0.2:
            objectivity = "Fairly balanced"
        else:
            objectivity = "Highly objective/factual"
        
        # Generate a brief explanation
        explanation = f"This video appears to be {sentiment.lower()} in tone and {objectivity.lower()} in content."
        
        # Identify key emotional words
        emotional_words = []
        for word, pos in analysis.tags:
            # Focus on adjectives, adverbs, and some verbs
            if pos.startswith('JJ') or pos.startswith('RB') or pos in ['VBD', 'VBG']:
                word_blob = TextBlob(word)
                if abs(word_blob.sentiment.polarity) > 0.3:
                    emotional_words.append(word)
        
        emotional_words = emotional_words[:5]  # Limit to top 5 emotional words
        
        # Create a user-friendly response
        result = {
            "sentiment": sentiment,
            "objectivity": objectivity,
            "explanation": explanation,
            "emotional_words": emotional_words if emotional_words else ["No strong emotional words detected"],
            "technical": {
                "polarity": round(analysis.polarity, 2),
                "subjectivity": round(analysis.subjectivity, 2)
            }
        }
        
        return result
        
    except Exception as e:
        # If TextBlob fails, fall back to a simpler pattern-matching approach
        logging.warning(f"Advanced sentiment analysis unavailable: {str(e)}. Using basic sentiment analysis instead.")
        
        # Define word lists for basic sentiment analysis
        positive_words = ["good", "great", "excellent", "amazing", "wonderful", "best", "love", 
                         "awesome", "fantastic", "positive", "beautiful", "helpful", "impressive",
                         "happy", "exciting", "recommend", "success", "better", "perfect"]
                         
        negative_words = ["bad", "terrible", "awful", "worst", "hate", "poor", "negative", 
                         "horrible", "disappointing", "failure", "problem", "difficult", "wrong",
                         "annoying", "useless", "boring", "waste", "worse", "ugly"]
                         
        # Convert text to lowercase for matching
        text_lower = summary_text.lower()
        
        # Count occurrences of positive and negative words
        positive_count = sum(1 for word in positive_words if re.search(r'\b' + word + r'\b', text_lower))
        negative_count = sum(1 for word in negative_words if re.search(r'\b' + word + r'\b', text_lower))
        
        # Determine sentiment based on word counts
        if positive_count > negative_count * 2:
            sentiment = "Very positive"
        elif positive_count > negative_count:
            sentiment = "Somewhat positive"
        elif negative_count > positive_count * 2:
            sentiment = "Very negative"
        elif negative_count > positive_count:
            sentiment = "Somewhat negative"
        else:
            sentiment = "Neutral"
        
        # Basic objectivity detection (looking for opinion words)
        opinion_words = ["think", "believe", "feel", "opinion", "seems", "appears", "suggest", 
                        "perhaps", "maybe", "might", "could", "should", "would", "claim"]
        
        opinion_count = sum(1 for word in opinion_words if re.search(r'\b' + word + r'\b', text_lower))
        fact_indicators = len(re.findall(r'\b\d+%?\b', text_lower))  # Numbers often indicate facts
        
        if opinion_count > fact_indicators * 2:
            objectivity = "Somewhat subjective"
        elif fact_indicators > opinion_count * 2:
            objectivity = "Fairly objective"
        else:
            objectivity = "Balanced mix of facts and opinions"
            
        explanation = f"This video appears to be {sentiment.lower()} in tone and {objectivity.lower()}."
        
        # Return simplified results
        return {
            "sentiment": sentiment,
            "objectivity": objectivity,
            "explanation": explanation,
            "note": "Basic sentiment analysis was used due to TextBlob initialization issues.",
            "technical": {
                "positive_words": positive_count,
                "negative_words": negative_count
            }
        }

In [41]:
print(analyze_sentiment(summary_text=summary_text))

{'polarity': 0.03144603934077619, 'subjectivity': 0.43737373737373736}


In [ ]:
print(analyze_sentiment2(summary_text=summary_text))

{'sentiment': 'Neutral', 'objectivity': 'Somewhat subjective', 'explanation': 'This video appears to be neutral in tone and somewhat subjective in content.', 'technical': {'polarity': 0.03, 'subjectivity': 0.44}}


: 